In [1]:
# Pandas and numpy necessary to do basic data cleaning
import pandas as pd
import numpy as np
import scipy.optimize

# Import and Clean Data

In [2]:
# Import data
df = pd.read_stata("./qje_data/Dataset_QJE_Replicate_with_Cities.dta")
df['_const'] = 1

# Replicate existence variable
df['exist1349']   = np.where((df['judaica'] == 1) | (df['comm1349'] == 1), 1, 0)

# Population controls
df['pop33']       = np.where(df['pop33'].isna(), df['n333pop'], df['pop33'])
df['pop245']      = np.where(df['n245pop'].isna(), (df['c25pop']/df['c33pop1'])*df['pop33'], df['n245pop'])
df['logpop25c']   = np.log(df['c25pop'])
df['logpop33c']   = np.log(df['c33pop1'])
df['logpop33_dep']= np.log(df['pop33'])
df['logpop245']   = np.log(df['n245pop'])
df['logpop285']   = np.log(df['n285pop'])
df['logpop309']   = np.log(df['n309pop'])
df['logpop333']   = np.log(df['n333pop'])
df['logpop1300']  = np.log(1000*df['pop_1300'])
df['logpop1500']  = np.log(1000*df['pop_1500'])
df['logpop1750']  = np.log(1000*df['pop_1750'])

# Number and share religion controls
df['perc_PROT25'] = 100*(df['c25prot']/df['c25pop'])
df['perc_CAT25']  = 100*(df['c25kath']/df['c25pop'])
df['perc_JEW25']  = 100*(df['c25juden']/df['c25pop'])
df['perc_JEW33']  = np.where((100*(df['jews33']/df['pop33'])).isna() | (100*(df['jews33']/df['pop33'])) == 1, 
                             df['perc_JEW25'], (100*(df['jews33']/df['pop33'])))
df['logjews25']   = np.log(1+df['c25juden'])
df['logjews33']   = np.log(1+df['jews33'])
df['logjews39']   = np.log(1+df['jews39'])

# Labor market controls
df['perc_Unemp33']  = 100*(df['c33erlos']/df['c33erwp'])
df['perc_Blue25']   = 100*(df['c25arbei']/df['c25berwt'])
df['perc_Blue33']   = 100*((df['c33arbei']+df['c33eloar'])/df['c33erwp'])
df['perc_Ag25']     = 100*(df['c25bland']/df['c25berwt'])
df['perc_Ag33']     = 100*(df['c33land']/df['c33erwtt'])
df['perc_Ind25']    = 100*(df['c25bwerk']/df['c25berwt'])
df['perc_Ind33']    = 100*(df['c33indu']/df['c33erwtt'])
df['perc_self25']   = 100*(df['c25selb']/df['c25berwt'])
df['perc_self33']   = 100*(df['c33selb']/df['c33erwtt'])
df['perc_RT25']     = 100*(df['c25bhand']/df['c25berwt'])
df['perc_RT33']     = 100*(df['c33hndl']/df['c33erwtt'])
df['perc_selfRT25'] = 100*(df['c25hselb']/df['c25bhand'])
df['perc_Ind25']    = np.where(df['perc_Ind25'].isna(), df['perc_Ind33'], df['perc_Ind25'])
df['perc_Blue25']   = np.where(df['perc_Blue25'].isna(), df['perc_Blue33'], df['perc_Blue25'])
df['perc_Ag25']     = np.where(df['perc_Ag25'].isna(), df['perc_Ag33'], df['perc_Ag25'])
df['perc_selfRT25'] = np.where(df['perc_selfRT25'].isna(), df['perc_self33'], df['perc_selfRT25'])
df['hephep_all']    = df['hephep']+df['hephep_instigated']

In [3]:
def get_XY_arrays(df,X_cols,Y_var,treat_var):
    
    # Exclude those with missing data
    reg_df = df[df['exist1349'] == 1]
    for var in [Y_var]+X_cols:
        reg_df = reg_df[~reg_df[var].isna()]
    reg_df.reset_index(drop=True, inplace=True)
    
    # Make XY arrays
    X          = np.array(reg_df[X_cols])
    Y          = np.array(reg_df[Y_var])[:,None]
    treatments = np.array(reg_df[treat_var])
    N          = X.shape[0]
    K          = X.shape[1]
    
    return reg_df,X,Y,treatments,N,K

In [4]:
X_cols    = ['pog1349','logpop25c','perc_JEW25','perc_PROT25','_const']
Y_var     = 'pog20s'
clust_var = 'kreis_nr'
treat_var = 'pog1349'

reg_df,X,Y,treatments,N,K = get_XY_arrays(df,X_cols,Y_var,treat_var)

# Q4(b) - Replication

### Panel A - Imputation Regression

In [5]:
def run_qje_regression(X,Y,reg_df,X_cols,Y_var,clust_var,N,K):
    
    # Get beta first
    beta = np.linalg.inv(X.T @ X) @ X.T @ Y
    
    # Build the 'meat' of the cluster sandwich SE estimator
    clust_cov_sum = np.zeros((K,K))
    for clust in np.sort(reg_df[clust_var].unique()):

        # Define data just from that cluster
        df_clust = reg_df[reg_df[clust_var] == clust]
        X_clust  = df_clust[X_cols].to_numpy()
        y_clust  = df_clust[Y_var].to_numpy()[:,None]

        # Do cluster robust SE formula
        u_j       = (y_clust - X_clust @ beta)
        clust_cov = X_clust.T @ u_j @ u_j.T @ X_clust
        clust_cov_sum += clust_cov

    # Get (X'X)^(-1): the 'bread' of the sandwich
    vcov = np.linalg.pinv(X.T @ X)

    # Finite-sample correction
    n_clust = reg_df[clust_var].unique().shape[0]
    qc      = (n_clust/(n_clust-1))*(N/(N-K))

    # Get standard errors of betas
    beta_SE = np.sqrt(np.diag(qc * vcov @ clust_cov_sum @ vcov))
    return beta.ravel(),beta_SE

In [6]:
# Get coefs and standard errors
beta,beta_SE = run_qje_regression(X,Y,reg_df,X_cols,Y_var,clust_var,N,K)

# Get adjusted R2
R2    = 1-np.sum((Y - X @ beta[:,None])**2)/np.sum((Y - np.mean(Y))**2)
adjR2 = 1-(1-R2)*((N-1)/(N-K))

### Panel B - Covariate Matching

In [7]:
# Functions for calculating pairwise obs distances
def make_dist_matrix(X_match, N, weight_type='inv_diag_vars'):

    # Get inverse diagonal variance matrix for weighting
    if weight_type == 'inv_diag_vars':
        variances = (np.sum((X_match-np.mean(X_match,axis=0))**2,axis=0))/N
        weighting = np.linalg.inv(np.diag(variances))
    elif weight_type == 'identity':
        weighting = np.eye(X_match.shape[1])
    
    # Populate distance matrix
    dist_matrix = np.zeros((N,N))
    for i in range(N):
        for j in range(N):

            # Set own-distance to infinty so it doesn't match to itself
            if i == j:
                dist_matrix[i,j] = np.inf

            # Set distance to obs with same treatment to infinity so they don't match
            elif treatments[i] == treatments[j]:
                dist_matrix[i,j] = np.inf

            # Otherwise allow distance calculation
            else:
                dist_matrix[i,j] = (
                    (X_match[i,:][:,None]-X_match[j,:][:,None]).T @ 
                    weighting @ 
                    (X_match[i,:][:,None]-X_match[j,:][:,None])
                )[0][0]
    return dist_matrix

In [8]:
def get_treatment_effect(X_match, dist_matrix, treatments, N_neighbors, N, treatment_params, return_neighbors=False):
    
    # Get K-th closest neighbor index and value
    nth_closest_idx = np.argsort(dist_matrix,axis=1)[:,N_neighbors-1]
    nth_closest_val = dist_matrix[np.arange(N), nth_closest_idx]
    
    # Find obs within distance of nth_closest_val and compute counterfactuals
    counterfactual = np.zeros(N)
    neighbors_idx_list =[]
    for i in range(N):
        neighbors_idx = np.argwhere(dist_matrix[i,:] <= nth_closest_val[i]).ravel()
        neighbors_idx_list.append(neighbors_idx)
        counterfactual[i] = np.mean(Y[neighbors_idx])
        
    # Calculate treatment effects
    treatment_dict = {}
    if 'att' in treatment_params:
        treated_idx = np.argwhere(treatments==1).ravel()
        treatment_dict['att'] = np.mean(Y[treated_idx]-counterfactual[treated_idx])
    if 'atu' in treatment_params:
        untreated_idx = np.argwhere(treatments==0).ravel()
        treatment_dict['atu'] = np.mean(counterfactual[untreated_idx]-Y[untreated_idx])
    if 'ate' in treatment_params:
        treated_idx   = np.argwhere(treatments==1).ravel()
        untreated_idx = np.argwhere(treatments==0).ravel()
        att = np.mean(Y[treated_idx]-counterfactual[treated_idx])
        atu = np.mean(counterfactual[untreated_idx]-Y[untreated_idx])
        treatment_dict['ate'] = np.mean(att*(treated_idx.shape[0]/N) + atu*(untreated_idx.shape[0]/N))
    
    if not return_neighbors:
        return treatment_dict
    else:
        return treatment_dict,neighbors_idx_list

In [9]:
def bootstrap_match_SE(X_match, dist_matrix, treatments, N_neighbors, N, treatment_params, N_boots):
    treatment_dict = {i:[] for i in treatment_params}
    for i in range(N_boots):
        
        # Sample observations with replacement and define new X
        new_idxs        = np.random.choice(np.arange(N), N)
        X_match_new     = X_match[new_idxs,:]
        dist_matrix_new = dist_matrix[new_idxs,:]
        dist_matrix_new = dist_matrix_new[:,new_idxs]
        
        # Get treatment effect for this new set
        treatment_dict_ind = get_treatment_effect(X_match_new, dist_matrix_new,
                                                  treatments, N_neighbors, N,
                                                  treatment_params)
        for i in treatment_params:
            treatment_dict[i].append(treatment_dict_ind[i])
    
    # Return standard deviation of estimators
    return {i:np.std(treatment_dict[i]) for i in treatment_params}

In [10]:
# Remove constant and treatment from X for matching procedures
X_matchB = np.array(reg_df[X_cols[1:-1]])

# Get results
dist_matrixB = make_dist_matrix(X_matchB, N)
attB         = get_treatment_effect(X_matchB, dist_matrixB, treatments, 4, N, ['att'])
att_seB      = bootstrap_match_SE(X_matchB, dist_matrixB, treatments, 4, N, ['att'], 4000)

### Panel C - Distance Matching

In [11]:
# Remove constant and treatment from X for matching procedures
X_matchC = np.array(reg_df[['Latitude','Longitude']])

# Get results
dist_matrixC = make_dist_matrix(X_matchC, N)
attC         = get_treatment_effect(X_matchC, dist_matrixC, treatments, 2, N, ['att'])
att_seC      = bootstrap_match_SE(X_matchC, dist_matrixC, treatments, 2, N, ['att'], 4000)

### Output Results to Latex

In [12]:
panel_a  = pd.DataFrame({"Panel":["Panel A: Regression Imputation","","",""],
                         "Result Name":["Pogrom 1349","Ln(Pop 1925)",
                                        "Pct Jewish 1925","Pct Protestant 1925"],
                         "Estimate":np.round(beta[:-1],4),
                         "Standard Error":np.round(beta_SE[:-1],4),
                         "Observations":[int(N),"","",""],
                         "Adjusted R2":[np.round(adjR2,4),"","",""]})
panel_b  = pd.DataFrame({"Panel":["Panel B: Demographic Matching"],
                         "Observations":[int(N)],
                         "Adjusted R2":[""],
                         "Result Name":["Pogrom 1349"],
                         "Estimate":[np.round(attB['att'],4)],
                         "Standard Error":[np.round(att_seB['att'],4)]})
panel_c  = pd.DataFrame({"Panel":["Panel C: Geographic Matching"],
                         "Observations":[int(N)],
                         "Adjusted R2":[""],
                         "Result Name":["Pogrom 1349"],
                         "Estimate":[np.round(attC['att'],4)],
                         "Standard Error":[np.round(att_seC['att'],4)]})
replication_df = pd.concat((panel_a,panel_b,panel_c))
replication_df.to_latex("./q4_replication_output/table6_replication.tex", index=False)

In [13]:
replication_df

,Panel,Result Name,Estimate,Standard Error,Observations,Adjusted R2
0,Panel A: Regression Imputation,Pogrom 1349,0.0607,0.0226,320,0.0544
1,,Ln(Pop 1925),0.0390,0.0152,,
2,,Pct Jewish 1925,0.0135,0.0114,,
3,,Pct Protestant 1925,0.0003,0.0004,,
0,Panel B: Demographic Matching,Pogrom 1349,0.0744,0.0192,320,
0,Panel C: Geographic Matching,Pogrom 1349,0.0819,0.0272,320,


# Q4(d)

### Run Regressions and Matching on Various Specifications

In [ ]:
X_cols1   = ['pog1349','logpop25c','perc_JEW25','perc_PROT25','perc_CAT25','_const']    # Add Catholics
X_cols2   = ['pog1349','logpop33_dep','perc_JEW33','perc_PROT25','perc_CAT25','_const'] # Change to '33 when possible
X_cols3   = ['pog1349','perc_JEW25','perc_PROT25','perc_CAT25','_const']                # Only religion variables
X_cols4   = ['pog1349','perc_Ag25','perc_Ind25','perc_Blue25','perc_self25','_const']   # Only labor variables
X_cols5   = ['pog1349','logpop1300','nav_river','ruggedness20']                         # Only geographic/predetermined
X_cols6   = ['pog1349','hephep_all',
             'logpop1300','nav_river','ruggedness20','logpop25c',
             'perc_JEW25','perc_PROT25','perc_CAT25',
             'perc_Ag25','perc_Ind25','perc_Blue25','perc_self25','_const']             # Fully loaded regression
X_cols    = [X_cols1,X_cols2,X_cols3,X_cols4,X_cols5,X_cols6]

# Loop over all specifications
betas         = []
beta_SEs      = []
Ns            = []
match_atts    = []
match_att_SEs = []
for i in range(len(X_cols)):
    
    # Get df and XY matrices and parameters
    reg_dfi,Xi,Yi,treatmentsi,Ni,Ki = get_XY_arrays(df,X_cols[i],Y_var,treat_var)
    Ns.append(Ni)
    
    # Run regressions and save output
    betai,beta_SEi = run_qje_regression(Xi,Yi,reg_dfi,X_cols[i],Y_var,clust_var,Ni,Ki)
    betas.append(betai)
    beta_SEs.append(beta_SEi)
    
    # Run matching procedure
    X_matchi       = np.array(reg_dfi[X_cols[i][1:-1]])
    dist_matrixi   = make_dist_matrix(X_matchi, Ni)
    atti           = get_treatment_effect(X_matchi, dist_matrixi, treatmentsi, 4, Ni, ['att'])
    att_sei        = bootstrap_match_SE(X_matchi, dist_matrixi, treatmentsi, 4, Ni, ['att'], 1000)
    match_atts.append(atti)
    match_att_SEs.append(att_sei)

### Output Results to Latex

In [ ]:
q4d_df  = pd.DataFrame({"Specification":["1 - Include Catholic Share",
                                         "2 - Change to 1933 Demographics",
                                         "3 - Only Religion Variables",
                                         "4 - Only Labor Variables",
                                         "5 - Only Predetermined Variables",
                                         "6 - Fully Loaded Regression",],
                         "Regression Estimate":np.round([betas[i][0] for i in range(len(betas))],4),
                         "Regression Standard Error":np.round([beta_SEs[i][0] for i in range(len(beta_SEs))],4),
                         "Matching ATT Estimate":np.round([match_atts[i]['att'] for i in range(len(match_atts))],4),
                         "Matching ATT Standard Error":np.round([match_att_SEs[i]['att'] for i in range(len(match_att_SEs))],4),
                         "Observations":[int(Ns[i]) for i in range(len(Ns))]})
q4d_df.to_latex("./q4_replication_output/q4d_table.tex", index=False)

In [ ]:
q4d_df

# Q4(e)

### Propensity-Score Matching

In [ ]:
# Covariates for this exercise
X_cols = ['pog1349','logpop25c','perc_JEW25','perc_PROT25','_const']
reg_df,X,Y,treatments,N,K = get_XY_arrays(df,X_cols,Y_var,treat_var)

# Get matrices for logistic regression
Xlogit = X[:,1:]
Ylogit = X[:,0][:,None]

In [ ]:
def logistic_log_likelihood(beta, Xlogit, Ylogit):
    return -np.sum(Ylogit*(Xlogit @ beta[:,None]) + np.log((1.0 / ( 1.0 + np.exp(Xlogit @ beta[:,None])))))

def logistic_gradient(beta, Xlogit, Ylogit):
    p = 1.0 / ( 1.0 + np.exp(- Xlogit @ beta[:,None]))
    return -(Xlogit.T @ (Ylogit-p)).ravel()

# Optimize logistic regression and get propensity scores
res = scipy.optimize.minimize(logistic_log_likelihood, x0=np.zeros(Xlogit.shape[1]), 
                              args=(Xlogit,Ylogit), method='BFGS', jac=logistic_gradient)
betalogit  = res.x
propscores = 1.0/(1.0 + np.exp(-(Xlogit @ betalogit[:,None])))
reg_df['propensity_scores'] = propscores

In [ ]:
# Now do propensity score matching
X_match     = np.array(reg_df['propensity_scores'])[:,None]
dist_matrix = make_dist_matrix(X_match, N, weight_type='identity')
treats      = get_treatment_effect(X_match, dist_matrix, treatments, 4, N, ['att','atu','ate'])
treat_ses   = bootstrap_match_SE(X_match, dist_matrix, treatments, 4, N, ['att','atu','ate'], 2500)

### Output Results to Latex

In [ ]:
propscore_df= pd.DataFrame({"Parameter":["ATT","ATU","ATE"],
                            "Estimate":np.round([treats['att'],treats['atu'],treats['ate']],4),
                            "Standard Error":np.round([treat_ses['att'],treat_ses['atu'],treat_ses['ate']],4)})
propscore_df.to_latex("./q4_replication_output/q4e_table.tex", index=False)

In [ ]:
propscore_df